<img src="https://raw.githubusercontent.com/UXARRAY/uxarray/main/docs/_static/images/logos/uxarray_logo_h_dark.svg"
     width="30%"
     alt="HEALPix logo"
     align="right"
/>

# UXarray for Basic HEALPix Statistics & Visualization

### In this section, you'll learn:

* Utilizing `intake` to open a HEALPix data catalog
* Using the `uxarray` package to look at basic statistics over HEALPix data
* Using UXarray plotting functionality on HEALPix data

### Related Documentation

* [UXarray homepage](https://uxarray.readthedocs.io/en/latest/index.html)
* [Working with HEALPix data - UXarray documentation](https://uxarray.readthedocs.io/en/latest/user-guide/healpix.html)
* [UXarray overview - Unstructured Grids Visualization Cookbook](https://projectpythia.org/unstructured-grid-viz-cookbook/notebooks/02-intro-to-uxarray/overview.html)
* [Data visualization with UXarray - Unstructured Grids Visualization Cookbook](https://projectpythia.org/unstructured-grid-viz-cookbook/notebooks/03-plotting-with-uxarray/data-viz.html)
* [Subsetting - UXarray documentation](https://uxarray.readthedocs.io/en/latest/user-guide/subset.html)
* [Intake Cookbook](https://projectpythia.org/intake-cookbook/README.html)

### Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [UXarray](https://uxarray.readthedocs.io/en/latest/index.html) | Necessary  | |
| [HEALPix overview](00-healpix) | Necessary  | |

**Time to learn**: 30 minutes

-----

In [ ]:
import cartopy.crs as ccrs
import uxarray as ux

## Open data catalog

:::{tip} If you think you first need to learn about Intake, Pythia's [Intake Cookbook](https://projectpythia.org/intake-cookbook/README.html) is a great resource to do so.:::

This time, let us use a different catalog than the previous section, [easy.gems for HEALPix Analysis & Visualization](01-easygems), and open the online catalog from the [WCRP's Digital Earths Global Hackathon 2025](https://digital-earths-global-hackathon.github.io/) using `intake` and read the output of the `ICON` run `ngc4008`, which is stored in the HEALPix format:

:::{tip} Despite using a different data catalog, this section will still showcase similar operations with the [previous section](01-easygems), e.g. basic statistics and global and regional data plotting, except at the end where further grid exploration methods will be demonstrated.:::

In [ ]:
import intake

# Final data catalog location (once hackathon website (https://digital-earths-global-hackathon.github.io/) updated)
# cat_url='https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml'
# Interim data catalog location
cat_url = "https://raw.githubusercontent.com/digital-earths-global-hackathon/catalog/refs/heads/ncar/online/main.yaml"
cat = intake.open_catalog(cat_url)
model_run = cat.icon_ngc4008

We will be looking at two resolution levels, one is the coarsest zoom level of 0, which is the default, and the other is finer at the zoom level of 7:

In [ ]:
ds_coarsest = model_run().to_dask()
ds_fine = model_run(zoom=7).to_dask()

### Create UXarray Datasets from HEALPix

Now let us use those `xarray.Dataset`s from the model run to open unstructured grid-aware `uxarray.UxDataset`:

In [ ]:
%%time
uxds_coarsest = ux.UxDataset.from_healpix(ds_coarsest)

uxds_fine = ux.UxDataset.from_healpix(ds_fine)
uxds_fine

## HEALPix basic stats using UXarray

Let us look at the global and Boulder, CO, USA air temperature averages for the dataset. Data spans from 2020 to 2050, so let us also consider slicing it to have something like 2020 to mid-2023, which would also give us similar results to that with easy.gems in the [previous section](01-easygems) (please note that they shouldn't be the exact same because of using different data catalogs).

In [ ]:
import matplotlib.pylab as plt

boulder_lon = -105.2747
boulder_lat = 40.0190

time_slice = slice("2020-01-02T00:00:00.000000000", "2023-07-31T00:00:00.000000000")

### Mesh face containing Boulder's coords

We can face(s) containing a given point with `uxarray` conveniently as follows:

In [ ]:
%%time
boulder_face = uxds_fine.uxgrid.get_faces_containing_point(
    point_lonlat=[boulder_lon, boulder_lat]
)

### Data variables of interest

In order to use in the rest of the analyses, we can grab data variables, `uxarray.UxDataArray`, from the dataset as follows:

In [ ]:
uxda_fine = uxds_fine.tas
uxda_coarsest = uxds_coarsest.tas

### Global and Boulder's temperature averages

In order to get a line plot of our `UXarray.UxDataset` objects' 1-dimensional temperature variables, we will convert them to `xarray` and call the default `plot` function because UXarray's default plot functions are all dedicated to grid-topology aware visualizations:

In [ ]:
%%time
uxda_fine.isel(n_face=boulder_face).sel(time=time_slice).to_xarray().plot(
    label="Boulder"
)
uxda_coarsest.sel(time=time_slice).mean("n_face").to_xarray().plot(label="Global mean")

plt.legend()

## Data plotting with UXarray

UXarray provides several built-in plotting functions to visualize unstructured grids, which can also be applied to HEALPix grids in the same interface:

Let us first look into interactive plots with `bokeh` backend (i.e. UXarray's plotting functions have a `backend` parameter that defaults to "bokeh", and it can also accept "matplotlib")

### Global plots

Let us first plot the global temperature (at the first timestep for simplicity), using the default backend, `bokeh`, of UXarray's visualization API to create an interactive plot:

In [ ]:
%%time

projection = ccrs.Robinson(central_longitude=-135.5808361)

uxda_fine.isel(time=0).plot(
    projection=projection,
    cmap="inferno",
    features=["borders", "coastline"],
    title="Global temperature",
    width=700,
)

Now, let us create the same plot, now using `matplotlib` as the backend:

In [ ]:
%%time

uxda_fine.isel(time=0).plot(
    backend="matplotlib",
    projection=projection,
    cmap="inferno",
    features=["borders", "coastline"],
    title="Global temperature",
    width=1100,
)

### Regional subsets (Not only for plotting but also for analysis)

When a region on the globe is of interest, UXarray provides subsetting functions, which return new regional grids that can then be used in the same way a global grid is plotted. 

Let us look into USA map using the Boulder, CO, USA coords we had used before for simplicity:

Subsetting `uxds_fine` into a new `UxDataset` using a "bounding box" around Boulder, CO first:

In [ ]:
%%time

lon_bounds = (boulder_lon - 20, boulder_lon + 40)
lat_bounds = (boulder_lat - 20, boulder_lat + 12)

uxda_fine_subset = uxda_fine.isel(time=0).subset.bounding_box(lon_bounds, lat_bounds)

If we quick check the global and regional subset's average temperature at the first timestep, we can see the difference:

In [ ]:
print(
    "Global temperature average: ", uxda_fine.isel(time=0).mean("n_face").values, " K"
)
print(
    "Regional subset's temperature average: ", uxda_fine_subset.mean("n_face").values, " K"
)

Now, let us plot the regional subset `UxDataset`:

In [ ]:
%%time
projection = ccrs.Robinson(central_longitude=boulder_lon)

uxda_fine_subset.plot(
    projection=projection,
    cmap="inferno",
    features=["borders", "coastline"],
    title="Boulder temperature",
    width=1100,
)

## Grid topology exploration

Exploring the grid topology may be needed sometimes, and UXarray provides functionality to do so, both numerically and visually. Each `UxDataset` or `UxDataArray` has their associated `Grid` object that has all the information such as spherical and cartesian coordinates, connectives, dimensions, etc. about the topology this data belongs to. This `Grid` object can be explored as follows:

In [ ]:
# uxds_fine.uxgrid   # this would give the same as the below
uxda_fine.uxgrid

There might be times that the user wants to open a standalone `Grid` object for a HEALPix grid (or any other unstructured grids supported by UXarray) without accessing the data yet. Let's create the coarsest HEALPix grid as follows:

In [ ]:
uxgrid = ux.Grid.from_healpix(zoom=0, pixels_only=False)
uxgrid

Let's investigate how a HEALPix grid looks like over the poles. We can do things by selecting an Orthographic projection and setting a Geodetic source projection. This allows each edge to be plotted as a great circle arc, which better approximates the true HEALPix structured compared to the defaut PlateCaree projection.


In [ ]:
projection = ccrs.Orthographic(central_latitude=90)
projection.threshold /= 100  # Smoothes out great circle arcs

uxgrid.plot(
    periodic_elements="ignore",  # Allow Cartopy to handle periodic elements
    crs=ccrs.Geodetic(),  # Enables edges to be plotted as GCAs
    project=True,
    projection=projection,
    width=500,
    title="HEALPix (Orthographic Proj), zoom=0",
)

Let's chose another projection.

In [ ]:
projection = ccrs.Mollweide()
projection.threshold /= 100  # Smoothes out great circle arcs

uxgrid.plot(
    periodic_elements="ignore",  # Allow Cartopy to handle periodic elements
    crs=ccrs.Geodetic(),  # Enables edges to be plotted as GCAs
    project=True,
    projection=projection,
    width=500,
    title="HEALPix (Mollweide Proj), zoom=0",
)

The grid structure here is approximated. While the boundary between each pixel is easily known in HEALPix workflows, UXarray represents the boundary as a Great Circle Arc (GCA) due to the requirement of having expcility connectivity information, which is slightly different and leads to minor differences in the computed plots.

Now that we've looked at the grid structured, we can also apply the same principles to our data plotting.

:::{tip} Using the Geodetic source projection is not reccomended for higher-resolution grids, as it introduces a significant overhead.:::

In [ ]:
uxda_coarsest.isel(time=0).plot(
    periodic_elements="ignore",  # Allow Cartopy to handle periodic elements
    crs=ccrs.Geodetic(),  # Enables edges to be plotted as GCAs
    project=True,
    projection=projection,
    features=["borders", "coastline"],
    cmap="inferno",
    title="Temperature (Mollweide Proj), zoom=0",
    width=500,
)

## What is next?

The [next section](03-uxarray-advanced) will provide an UXarray workflow that loads in and analyzes & visualizes HEALPix data.